In [ ]:
# This script is to derive absolute date features from the raw_target_date field for each article row.

In [52]:
from datetime import datetime, timedelta, date
import pandas as pd
import spacy
import en_core_web_sm
from spacy.matcher import Matcher
nlp = en_core_web_sm.load()

In [53]:
newsdf = pd.read_csv('Hindu-Scraper/newsAprLabelledWithActors.csv', delimiter='|')
newsdf[newsdf['is_event'] == 1]

,title,location,date,body,keywords,output,target_date,actors,is_event,raw_target_date
5,Cases booked for violation of code,THANJAVUR,"April 01, 2019 00:00 IST",The Tamil University police have booked cases ...,civil supplies corporation employees associati...,1,NaN,"['Tamil University', 'AIADMK', 'Amma Makkal', ...",1,April 1
53,Maoists call for bandh in Vizag Agency on April 5,VISAKHAPATNAM,"April 01, 2019 00:00 IST",The banned CPI (Maoist) has called for a bandh...,march 15 near buradamamidi village using repre...,1,NaN,"['the Visakha Agency', 'anti-Naxal', 'Greyhoun...",1,April 5
90,Pak. envoy made Foreign Secretary,NEW DELHI,"April 01, 2019 00:00 IST","Sohail Mahmood, the current High Commissioner ...",foreign minister shah mahmood qureshi announce...,1,NaN,"['Sohail Mahmood', 'Pakistan', 'India', 'Islam...",1,the middle
117,Maoists call for bandh on April 5,VISAKHAPATNAM,"April 01, 2019 00:00 IST",The banned CPI (Maoist) have called for a band...,march 15 near buradamamidi village koraput – v...,1,NaN,"['Agency', 'Visakhapatnam', 'anti-Naxal', 'Gre...",1,April 5
138,‘Vote Run’ attracts large numbers,MANGALURU,"April 01, 2019 00:00 IST","People in large numbers, including children fr...",cycling group members also took part marathon ...,0,NaN,"['the Systematic Voters Education', 'Electoral...",1,less than
170,Cong. activists hail Rahul’s candidature,Kozhikode,"April 01, 2019 00:00 IST",The belated declaration of Congress national p...,wayanad lok sabha constituency brought back ch...,0,NaN,"['Congress', 'Rahul Gandhi’s', 'Kozhikode', 'M...",1,NaN
217,Fishermen anxious about frequent mid-sea accid...,KOCHI,"April 01, 2019 00:00 IST",The Saturday morning mid-sea accident in which...,shipping channel kerala swathanthra matsya tho...,1,NaN,"['Munambam', 'Kerala Swathanthra', 'Thozhilali...",1,NaN
223,Film-shoot tragedy: Family of deceased stages ...,Bengaluru,"April 01, 2019 00:00 IST",Relatives of 29-year-old Sumera and her five-y...,"director samudram v ., production manager kira...",1,NaN,"['Sumera', 'Ahera', 'Kannada', 'Bagalur', 'Tab...",1,Friday
521,Fishermen stage protest for registration of boats,Thoothukudi,"April 02, 2019 00:00 IST",Around 150 people belonging to the fishing com...,tamil nadu marine fisheries regulation act mec...,1,NaN,"['M. Samy', 'Mercantile Marine']",1,45
546,Two independents stage protest near offices of...,Virudhunagar,"April 02, 2019 00:00 IST","Two Independent candidates, N. Vallinayagam, f...",respective returning officers complaining viru...,1,NaN,"['N. Vallinayagam', 'Virudhunagar Lok Sabha', ...",1,daily


In [54]:
newsdf['date_diff'] = ''

In [55]:
def getDayNumber(day):
    day = day.lower()
    if day == 'monday':
        return 0
    elif day == 'tuesday':
        return 1
    elif day == 'wednesday':
        return 2
    elif day == 'thursday':
        return 3
    elif day == 'friday':
        return 4
    elif day == 'saturday':
        return 5
    elif day == 'sunday':
        return 6
    else:
        return -1    

In [56]:
# Calendrical arithmetic for day of the week
def roundSub(cur, target):
    if target < cur:
        return 6 + (target - cur)
    else:
        return target - cur
# returns absolute date from raw_target_dates which are usually relative temporal units
# returns 'PAST' - If raw_target_date refers to a date before published date
# returns 'UNKNOWN' - If unable to parse or derive
def calculateDate(pdate, raw_target):
    if type(raw_target) is not str:
        return 'UNKNOWN'
    raw_target = raw_target.lower()
    raw_no = getDayNumber(raw_target)
    if raw_no != -1:
        if raw_no == pdate.weekday():
            return pdate
        else:
            return pdate + timedelta(days=roundSub(pdate.weekday(), raw_no))
    try:
        month,day = raw_target.split()
        mno = datetime.strptime(month, '%B').month
        dno = datetime.strptime(day, '%d').day
        target = date(2019, mno, dno)
        if target < pdate:
            return 'PAST'
        else:
            return target
    except:
        return 'UNKNOWN'

In [57]:
newsdf['target_date'] = None
for index,row in newsdf.iterrows():
    pdate = datetime.strptime(row.get('date').strip('IST').strip(),'%B %d, %Y %H:%M').date()
    target = calculateDate(pdate, row.get('raw_target_date'))
    newsdf.at[index,'target_date'] = target
    if type(target) is date:
        newsdf.at[index, 'date_diff'] = (target-pdate).days
print('done')

done


In [60]:
for index,row in newsdf.iterrows():
    doc = nlp(row.get('body'))
    newsdf.at[index, 'actors'] = [e.text for e in doc.ents if e[0].ent_type_ in ['ORG', 'GPE', 'PERSON']]
print('Done')
newsdf

Done


,title,location,date,body,keywords,output,target_date,actors,is_event,raw_target_date,date_diff
0,SC suspends eco clearance for international ai...,NEW DELHI,"April 01, 2019 00:00 IST",The health of the environment is key to preser...,"transparent institutions ,” justice chandrachu...",0,UNKNOWN,"[the Supreme Court, Goa, Hemant Gupta, Justice...",0,NaN,
1,‘Bid to dislodge AIADMK from T.N. won’t succeed’,Vellore,"April 01, 2019 00:00 IST",Deputy Chief Minister O. Panneerselvam has c...,‘ two leaves ’ symbol vellore lok sabha candid...,0,UNKNOWN,"[O. Panneerselvam, DMK, M.K. Stalin, A.C. Shan...",0,NaN,
2,Centre turns down Bengal’s request for paramil...,Kolkata,"April 01, 2019 00:00 IST",The Mamata Banerjee government’s plea to ret...,000 paramilitary personnel posted mamata baner...,0,UNKNOWN,"[Mamata Banerjee, ’s, Central, Communist Party...",0,NaN,
3,Prayers before polls,Tirunelveli,"April 01, 2019 00:00 IST",Whenever she happened to pass by the Kotturp...,late chief minister jayalalithaa ’ roman catho...,0,UNKNOWN,"[the Kotturpuram Vinayakar Temple, Chennai, Ja...",0,NaN,
4,‘Christian missionaries’ contribution to Tamil...,MADURAI,"April 01, 2019 00:00 IST",The contribution of Christian missionaries to ...,tamil nadu bishops ’ council antony pappusamy ...,0,UNKNOWN,"[P. Nedumaran, Tamil Desiya Iyakkam, St. Josep...",0,NaN,
5,Cases booked for violation of code,THANJAVUR,"April 01, 2019 00:00 IST",The Tamil University police have booked cases ...,civil supplies corporation employees associati...,1,2019-04-01,"[Tamil University, AIADMK, Amma Makkal, Munnet...",1,April 1,0
6,Tipparaju Hawaldar’s supporters angry,RAICHUR,"April 01, 2019 00:00 IST",The row over the denial of ticket from the Ra...,former raichur rural mla tipparaju hawaladar r...,0,UNKNOWN,"[Raichur Rural MLA Tipparaju Hawaladar, the Bh...",0,NaN,
7,"BJP, a betrayer: Mutharasan",THANJAVUR,"April 01, 2019 00:00 IST","R.Mutharasan, State secretary, Communist Party...",tamil nadu state leader claims thanjavur lok s...,0,UNKNOWN,"[R.Mutharasan, State, Communist Party of India...",0,NaN,
8,31brief2,Vijayawada,"April 01, 2019 00:00 IST",BJP State president Kanna Lakshminarayana se...,ap planning board vice chairman c bjp state pr...,0,UNKNOWN,"[BJP State, Kanna Lakshminarayana, AP Planning...",0,NaN,
9,Water level,Tirunelveli,"April 01, 2019 00:00 IST",Water level in Papanasam dam on Sunday stood...,sunday stood papanasam dam manimuthar dam 75 c...,0,UNKNOWN,"[Papanasam, Manimuthar]",0,NaN,


In [61]:
newsdf.to_json('Hindu-Scraper/newsAprFullFeaturedA.json', orient='records', date_format='iso')